In [37]:
import regex as re
text = open("tests/fixtures/tinystories_sample_5M.txt", "r", encoding="utf-8").read()

# split corpus on special tokens
special_pat = "|".join(map(re.escape, ['<|endoftext|>']))

parts = re.split(special_pat, text)

text = "".join(parts)

In [57]:
text = open("tests/fixtures/tinystories_sample_5M.txt", "r", encoding="utf-8").read()

In [58]:
text = 'a<|endoftext|>' + text + '<|endoftext|>a'

In [59]:
text[-100:]

' They cleaned the rocks together and made them shiny.\nAfter cleaning the rocks, Bob an<|endoftext|>a'

In [60]:
parts = re.split(special_pat, text)

In [61]:
parts[:10]

['a',
 'u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could

In [3]:
import os
import time
from new_bpe import RegexTokenizer

# create a directory for models, so we don't pollute the current directory
os.makedirs("models", exist_ok=True)

t0 = time.time()

# construct the Tokenizer object and kick off verbose training
tokenizer = RegexTokenizer(special_tokens={'<|endoftext|>': 999})
tokenizer.train(text, 256 + 100, verbose=True)
# writes two files in the models directory: name.model, and name.vocab
#prefix = os.path.join("models", "regex")
#tokenizer.save(prefix)
t1 = time.time()

print(f"Training took {t1 - t0:.2f} seconds")

merge 1/99: (104, 101) -> 256 (b'he') had 148935 occurrences
merge 2/99: (32, 116) -> 257 (b' t') had 148857 occurrences
merge 3/99: (32, 97) -> 258 (b' a') had 111510 occurrences
merge 4/99: (32, 115) -> 259 (b' s') had 76372 occurrences
merge 5/99: (32, 119) -> 260 (b' w') had 74350 occurrences
merge 6/99: (110, 100) -> 261 (b'nd') had 67904 occurrences
merge 7/99: (257, 256) -> 262 (b' the') had 67683 occurrences
merge 8/99: (101, 100) -> 263 (b'ed') had 58392 occurrences
merge 9/99: (32, 98) -> 264 (b' b') had 52390 occurrences
merge 10/99: (257, 111) -> 265 (b' to') had 49394 occurrences
merge 11/99: (258, 261) -> 266 (b' and') had 45801 occurrences
merge 12/99: (32, 104) -> 267 (b' h') had 42014 occurrences
merge 13/99: (32, 102) -> 268 (b' f') had 39838 occurrences
merge 14/99: (105, 110) -> 269 (b'in') had 38758 occurrences
merge 15/99: (260, 97) -> 270 (b' wa') had 38433 occurrences
merge 16/99: (32, 84) -> 271 (b' T') had 38135 occurrences
merge 17/99: (105, 116) -> 272 (b'it

In [5]:
import pickle

In [6]:
p = "tests/_snapshots/test_train_bpe_special_tokens.pkl"

with open(p, 'rb') as f:
    data = pickle.load(f)

In [9]:
p = "new9.pkl"

with open(p, 'rb') as f:
    data2 = pickle.load(f)

In [10]:
for i, (a, b) in enumerate(zip(data["merges"], data2["merges"])):
    if a != b:
        print(f"Mismatch at idx{i}: ground_truth = {a}, my_merges = {b}")

In [ ]:
from collections import Counter
from itertools import pairwise

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
tok_pat = re.compile(pattern=PAT)
input_path = "tests/fixtures/tinystories_sample_5M.txt"
BLOCK = 64 * 1024 * 1024 # chunk block size
pair_counts = Counter()
words = []

with open(input_path, "r", encoding="utf-8") as f:
    tail = ""

    while True:
        chunk = f.read(BLOCK)
        data = tail + chunk if chunk else tail # append unused tail
        if not data:
            break
        parts = re.split(special_pat, data) # split data into parts using special_tokens as a delimiter
        if len(parts) > 1:
            tail = parts.pop()  # assign tail to the last part as it might be cutoff
        else:
            tail = ""

        # parts is a list of the split sections. 
        # iterate through each doc
        for doc in parts:
            # iterate through every match found using the pretokenization pattern
            for match in re.finditer(pattern=tok_pat, string=doc):
                # prev_id = None
                ids = [bytes([b]) for b in match.group().encode("utf-8")]  # list of bytes in the encoded token 'hello' -> [b'h',b'e',b'l',b'l',b'o']
                words.append(ids)  # append each list of bytes (ids) to the words list. the words list holds all the splits from the regex pattern
                pair_counts.update(pairwise(ids))  # update counter with pairs from each match. same as looping through with zip(ids, ids[1:])
        print(len(words))

5240722
1263005
0
1263131
0


In [20]:
parts

['\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the rocks together and made them shiny.\nAfter cleaning the rocks, Bob an\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the

In [21]:
doc

'\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the rocks together and made them shiny.\nAfter cleaning the rocks, Bob an\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the 

In [12]:
pair_counts.most_common(4)

[((b'h', b'e'), 148950),
 ((b' ', b't'), 148875),
 ((b' ', b'a'), 111524),
 ((b't', b'h'), 101585)]

In [31]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
tok_pat = re.compile(pattern=PAT)
input_path = "tests/fixtures/tinystories_sample_5M.txt"
BLOCK = 64 * 1024 * 1024
pair_countss = Counter()
words2 = []

with open(input_path, "r", encoding="utf-8") as f:
    buffer = ""
    
    while True:
        chunk = f.read(BLOCK)
        if not chunk:
            # Process remaining buffer
            if buffer:
                data = buffer
            else:
                break
        else:
            data = buffer + chunk
            
        # Split by special tokens
        parts = re.split(special_pat, data)
        
        if chunk:  # If not end of file
            # Keep the last part as buffer (might be incomplete)
            buffer = parts.pop()
        else:
            buffer = ""
            
        # Process complete parts
        for doc in parts:
            for match in re.finditer(pattern=tok_pat, string=doc):
                ids = [bytes([b]) for b in match.group().encode("utf-8")]
                words2.append(ids)
                pair_countss.update(pairwise(ids))
        print(len(words2))

1263005
1263131


In [28]:
len(words)

1263257

In [29]:
len(words2)

1263131

In [23]:
doc

'\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the rocks together and made them shiny.\nAfter cleaning the rocks, Bob an'

In [25]:
parts

['\nOnce upon a time, there was a messy giant named Bob. Bob lived in a big forest with many trees. Bob was always making a mess because he was so big and clumsy. One day, Bob decided he wanted to sell his big rocks to the people in the town.\nBob walked to the town with a big bag of rocks. He met a little girl named Sue. Sue looked at the rocks and said, "These rocks are very messy, but I can help you clean them." Bob was happy and thanked Sue. They cleaned the rocks together and made them shiny.\nAfter cleaning the rocks, Bob an']

In [14]:
pair_counts.most_common(4)

[((b'h', b'e'), 148935),
 ((b' ', b't'), 148857),
 ((b' ', b'a'), 111510),
 ((b't', b'h'), 101572)]